In [1]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
from app.classes.data_processor import DataProcessor
from app.classes.table_creator import TableCreator
from app.classes.table_merger import TableMerger
from app.classes.table_processor import TableProcessor
from app.utils.save_file import save_local_file_xlsx, save_daily_allocation_OV, save_local_file_csv
from app.utils.treat_table import format_float_2_decimal

file_content_billing = './data/files/billing_hp_2024.csv'
file_content_general_data = './data/files/general_data.xlsx'
file_content_ibge = './data/files/populacao_municipios_ibge_2022.xlsx'

data_billing_ytd = DataProcessor.type_columns(file_content_general_data, sheet_name="billing_hp_2024").data
data_billing_ay = DataProcessor.type_columns(file_content_general_data, sheet_name="Billing").data

data_billing = DataProcessor.concat_table_billing(data_billing_ytd, data_billing_ay).data

data_customers = DataProcessor.type_columns(file_content_general_data, sheet_name="Customers").data
data_products = DataProcessor.type_columns(file_content_general_data, sheet_name="Products").data
data_volume = DataProcessor.type_columns(file_content_general_data, sheet_name="Volume").data
data_pending = DataProcessor.type_columns(file_content_general_data, sheet_name="OV").data
data_volume_reg = DataProcessor.type_columns(file_content_general_data, sheet_name='Volume').data
data_stock = DataProcessor.type_columns(file_content_general_data, sheet_name='Stock').data
data_frete = DataProcessor.type_columns(file_content_general_data, sheet_name='frete_cidade').data
data_ibge = DataProcessor.type_columns(file_content_ibge, sheet_name='Municipios').data

In [3]:
table_main = TableCreator.create_table_main(data_customers, data_products).data
table_customers = TableCreator.create_table_customers(data_customers).data
table_products = TableCreator.create_table_products(data_products).data
table_billing_ytd = TableCreator.create_table_billing_ytd(data_billing).data
table_pareto_customers = TableCreator.create_table_pareto_customers(table_billing_ytd).data
table_volume_sku = TableCreator.create_table_volume_sku(data_volume).data
table_pareto_private_customers = TableCreator.create_table_pareto_private_customers(table_billing_ytd, table_customers).data
table_pareto_products = TableCreator.create_table_pareto_products(table_billing_ytd).data
table_portfolio = TableCreator.create_table_portfolio(table_billing_ytd, table_pareto_products).data
table_billing_customers_sba = TableCreator.create_table_billing_customers_sba(table_billing_ytd).data
table_billing_customers_hospital_care = TableCreator.create_table_billing_customers_hospital_care(table_billing_ytd).data
table_purchase_frequency = TableCreator.create_table_purchase_frequency(data_billing).data
table_purchase_frequency_customers = TableCreator.create_table_purchase_frequency_customers(data_billing).data
table_last_month_purchase = TableCreator.create_table_last_month_purchase(data_billing).data
table_billing_ay = TableCreator.create_table_billing_actual_year(data_billing_ay).data
table_pending_customers = TableCreator.create_table_pending_customers(data_pending).data
table_representation_sales_sba = TableCreator.create_table_representation_sales_sba(data_billing).data
table_unit_price = TableCreator.create_table_unit_price(data_billing).data
table_pending = TableCreator.create_table_pending(data_pending).data
table_stock = TableCreator.create_table_stock(data_stock).data
table_volume_reg = TableCreator.create_table_volume_reg(data_volume_reg).data
table_billing_am = TableCreator.create_table_billing_actual_month(data_billing_ay).data
table_frete = TableCreator.create_table_freight(data_billing_ay, data_ibge).data
table_alert_limits = TableCreator.create_table_alert_limits(data_billing_ay, table_frete).data

In [4]:
table_alert_limits.head()

,Cidade,UF,ICF_Historico_%,Limite_Alerta_%
0,Rio Branco,AC,0.000000,0.000000
1,Maceio,AL,0.087018,0.121825
2,Sao Miguel Dos Campos,AL,0.000000,0.000000
3,Manaus,AM,0.479114,0.670759
4,Tabatinga,AM,0.000000,0.000000


In [5]:
table_general_merge = TableMerger.merge_table_general(
    table_main,
    table_billing_ytd,
    table_customers,
    table_products,
    table_volume_sku,
    table_pareto_customers,
    table_pareto_private_customers,
    table_pareto_products,
    table_portfolio,
    table_billing_customers_sba,
    table_billing_customers_hospital_care,
    table_purchase_frequency,
    table_purchase_frequency_customers,
    table_last_month_purchase,
    table_billing_ay,
    table_pending_customers,
    table_representation_sales_sba,
    table_stock,
    table_billing_am,
    table_unit_price,
    filter=['IV FLUIDS & IRRIGATION', 'DRUGS', 'PARENTERAL NUTRITION']
).data

In [6]:
table_general_params = TableProcessor.create_params(table_general_merge).data
table_general = TableProcessor.create_coefficient_normalized(table_general_params).data

In [7]:
table_allocation_merge = TableMerger.merge_table_allocation(
    table_pending,
    table_general,
    table_customers,
    table_billing_am,
    table_volume_reg,
    table_stock,
    table_frete,
    table_alert_limits
).data

In [8]:
table_allocation_rules = TableCreator.create_table_business_rules(table_allocation_merge).data

In [9]:
table_allocation = TableProcessor.allocate(table_allocation_rules).data

table_allocation_treated = TableProcessor.treat_allocation_table(table_allocation).data

In [10]:
table_allocation_minimum_order = TableProcessor.create_minimum_order(table_allocation_treated).data

In [11]:
format_float_2_decimal(table_general, 2)
format_float_2_decimal(table_allocation_minimum_order, 2)

In [13]:
save_local_file_xlsx(file=table_allocation_minimum_order, name="table_allocation_rules")

Save: ./data/files/table_allocation_rules.xlsx
